In [1]:
import sys # biblioteca para controlar a execução do programa
import vlc # Importa o VLC media player, necessário para a execução do video

# PyQt5.QtWidgets contém os elemenos da interface gráfica, como botões, janelas e sliders
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QFileDialog, QSlider, QWidget, QVBoxLayout, QLabel
# Biblioteca necessárias para configuração e atualização da barra de rolagem
from PyQt5.QtCore import Qt, QTimer

In [2]:
# Classe responsável pela criação de uma janela separada para a exibição do vídeo
class VideoWindow(QWidget):
    
    # Construtor da classe
    def __init__(self):

        # super() é usado para chamar métodos da classe pai (a classe da qual a atual herda, que no caso é a QWidget).
        # Neste caso, super().__init__() chama o construtor da classe QWidget, garantindo que qualquer inicialização 
        # necessária da classe base ocorra antes de qualquer outra modificação. No caso da interface gráfica, esse 
        # comando garante que todas as configurações padrão da janela sejam carregadas antes de adicionarmos 
        # elementos personalizados.
        super().__init__()

        self.setWindowTitle("Vídeo") # Define o título da janela como 'Video'
        self.setGeometry(300, 100, 800, 500)  # Define o tamanho inicial da janela de vídeo

# Classe responsável pela criação de uma janela específica para os controles do player
class ControlWindow(QMainWindow):
    
    # Construtor da classe
    def __init__(self):

        super().__init__() # realiza a chamada dos métodos da classe pai (análogo à classe VideoWindow())

        self.setWindowTitle("Controles do Player") # Define o título da janela como 'Controles do Player'
        self.setGeometry(100, 100, 400, 200) # Define o tamanho inicial da janela de controles

        # É criado um widget central para agrupar os botões de controle, bem como a barra
        # de rolagem do vídeo. Além disso o layout do widget é definido (adotando um estilo
        # vertical para permitir que os elementos seja empilhados um sobre o outro).

        self.central_widget = QWidget(self) # Criação da instância do widget central
        self.setCentralWidget(self.central_widget) # Definindo o objeto como o widget central
        self.layout = QVBoxLayout() # Definindo uma instância do layout vertical
        self.central_widget.setLayout(self.layout) # Associando o layout para o widget central

        # Definição elementos de controle (botões e slider)

        self.slider = QSlider(Qt.Horizontal) # slider
        self.open_button = QPushButton("Abrir Vídeo") # botão para abrir vídeo
        self.play_button = QPushButton("Play/Pause") # botão para dar play ou pause

        # Inserindo os elementos criados na janela de controles
        # observação: A ordem importa para a visualização

        self.layout.addWidget(self.open_button)
        self.layout.addWidget(self.play_button)
        self.layout.addWidget(self.slider)
    
        # Criação de Label para exibir o tempo de vídeo

        self.time_label = QLabel("00:00 / 00:00") # Instância de um label (formato "00:00 / 00:00")
        self.layout.addWidget(self.time_label) # Inserção do label como elemento da janela de controles

        # Criação da Janela do Vídeo, sendo ela uma intância 
        # da classe VideoWindow() criada préviamente, além de 
        # associar o VLC à janela criada.

        self.video_window = VideoWindow() # Instância da janela
        self.video_window.show() # Exibir a janela instanciada (sem conteúdo)

        self.instance = vlc.Instance() # cria uma instância do framework do VLC
        self.media_player = self.instance.media_player_new() # cria um novo player de vídeo a partir da instância do vlc
        self.media_player.set_hwnd(int(self.video_window.winId())) # Vincula o player do VLC à janela criada

        # Associa as funções de controle aos botões criados 

        self.open_button.clicked.connect(self.open_file) # Abrir Arquivo
        self.play_button.clicked.connect(self.toggle_play_pause) # Play/Pause do vídeo
        self.slider.sliderMoved.connect(self.set_position) # Define a posição da barra de rolagem

        # Timer responsável por atualizar a barra de progresso

        self.timer = QTimer(self) # Cria uma instância do temporizador
        self.timer.setInterval(500) # Define um intervalo de tempo de 500ms
        self.timer.timeout.connect(self.update_slider) # O slider é atualizado a cada 500ms, utilizando a fução update

    # Função responsável por permitir a escolha de um arquivo utilizando o explorer 
    def open_file(self):

        # QFileDialog.getOpenFileName() abre uma janela de seleção de aquivos, retornando dois valores:
        # 1 - O caminho do arquivo selecionado (exemplo: "C:/Videos/filme.mp4").
        # 2 - Um valor extra que contém o filtro de arquivos aplicado

        # Parâmetros utilizados
        # self - Referência à janela principal
        # "Abrir Vídeo" - Título da janela de seleção de arquivos.
        # "" - Diretório inicial (se vazio, abre no último local acessado).
        # "Arquivos de Vídeo (*.mp4 *.avi *.mkv)" - Filtro para exibir apenas arquivos de vídeo.

        file_name, _ = QFileDialog.getOpenFileName(self, "Abrir Vídeo", "", "Arquivos de Vídeo (*.mp4 *.avi *.mkv)")
        
        # Valida se existe um arquivo de vídeo selecionado

        if file_name:

            media = self.instance.media_new(file_name) # Cria um objeto de mídia a parir da instância do  VLC e associa o vídeo selecionado
            self.media_player.set_media(media) # O objeto de mídia (media) é atribuído ao player de vídeo criado pela instância do VLC
            self.media_player.play() # O vlc inicia a reprodução do vídeo carregado
            self.timer.start() # Inicia o timer da barra de progresso

    def toggle_play_pause(self):

        # Valida se o vídeo está reproduzindo
        # Em caso positivo, o vídeo é pausado. Em caso negativo o vídeo é reproduzido

        if self.media_player.is_playing():

            self.media_player.pause()

        else:

            self.media_player.play()

    # Define a posição do vídeo quanto ao slider
    def set_position(self, position):

        if self.media_player.get_length() > 0: # Valida se o tamanho do vídeo é superior a 0

            self.media_player.set_position(position / 1000.0) # A posição do vídeo é definida de acordo com a variável position definida em update_slider

    # Atualiza a posição do slider
    def update_slider(self):

        if self.media_player.get_length() > 0: # Valida se o tamanho do vídeo é superior a 0

            position = self.media_player.get_position() * 1000 # A posição é dada pela posição do objeto de mídia criado
            self.slider.setValue(int(position)) # O valor da posição do slider recebe a posição do objeto de vídeo
            elapsed_time = self.format_time(self.media_player.get_time() // 1000) # Define o tempo de vídeo formatado
            total_time = self.format_time(self.media_player.get_length() // 1000) # Define o tempo total de vídeo formatado
            self.time_label.setText(f"{elapsed_time} / {total_time}") # Define a exibição do tempo na janela
    
    # Formatação do tempo como (mm:ss)
    def format_time(self, seconds):

        minutes = seconds // 60
        seconds = seconds % 60
        return f"{minutes:02}:{seconds:02}"

In [4]:
app = QApplication(sys.argv)
player = ControlWindow()
player.show()
sys.exit(app.exec())

: 